In [56]:
!mkdir hugeCTR_saved_model

config = """{
  "solver": {
    "lr_policy": "fixed",
    "display": 1000,
    "max_iter":20000,
    "gpu": [0],
    "batchsize": 16384,
    "snapshot": 1000,
    "snapshot_prefix": "./hugeCTR_saved_model",
    "eval_interval": 5000,
    "eval_batches": 100,
    "mixed_precision": 1024,
    "eval_metrics": ["AUC:0.999"]
  },
  
  "optimizer": {
    "type": "SGD",
    "global_update": false,
    "sgd_hparam": {
    "learning_rate": 0.1,
      "warmup_steps": 1000
    }
  },


  "layers": [ 
      {
      "name": "data",
      "type": "Data",
      "slot_size_array": [138493 , 26744],
      "slot_size_array_orig": [138493 , 26744],
      "source": "./data/hugeCTR/filelist.txt",
      "eval_source": "./data/hugeCTR/test_filelist.txt",
      "check": "None",
      "cache_eval_data": true,
      "label": {
              "top": "label",
              "label_dim": 1
      },
      "dense": {
              "top": "dense",
              "dense_dim": 1
      },
      "sparse": [
              {
          "top": "data1",
          "type": "LocalizedSlot",
          "max_feature_num_per_sample": 2,
          "max_nnz": 1,
          "slot_num": 2
              }
      ]
    },

    {
      "name": "sparse_embedding1",
      "type": "LocalizedSlotSparseEmbeddingOneHot",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "slot_size_array": [138493 , 26744],
        "embedding_vec_size": 128,
        "combiner": 0
      }
    },

    {
      "name": "fc1",
      "type": "FusedInnerProduct",
      "bottom": "dense",
      "top": "fc1",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc2",
      "type": "FusedInnerProduct",
      "bottom": "fc1",
      "top": "fc2",
      "fc_param": {
        "num_output": 256
      }
    },

   
    {
      "name": "fc3",
      "type": "FusedInnerProduct",
      "bottom": "fc2",
      "top": "fc3",
      "fc_param": {
        "num_output": 128
      }
    },

    {
      "name": "interaction1",
      "type": "Interaction",
      "bottom": ["fc3", "sparse_embedding1"],
      "top": "interaction1"
    },

    {
      "name": "fc4",
      "type": "FusedInnerProduct",
      "bottom": "interaction1",
      "top": "fc4",
       "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc5",
      "type": "FusedInnerProduct",
      "bottom": "fc4",
      "top": "fc5",
      "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc6",
      "type": "FusedInnerProduct",
      "bottom": "fc5",
      "top": "fc6",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc7",
      "type": "FusedInnerProduct",
      "bottom": "fc6",
      "top": "fc7",
      "fc_param": {
        "num_output": 256
      }
    },

    {
      "name": "fc8",
      "type": "InnerProduct",
      "bottom": "fc7",
      "top": "fc8",
      "fc_param": {
        "num_output": 1
      }
    },
    
    {
      "name": "loss",
      "type": "BinaryCrossEntropyLoss",
      "bottom": ["fc8","label"],
      "top": "loss"
    } 
  ]
}
"""

mkdir: cannot create directory 'hugeCTR_saved_model': File exists


In [57]:
with open('./movie_lens.json', 'wt') as f:
    for line in config:
        f.write(line)

In [58]:
!../build/bin/huge_ctr --train ./movie_lens.json

[0, init_start, ]
HugeCTR Version: 2.2
Config file: ./movie_lens.json
[10d05h56m42s][HUGECTR][INFO]: batchsize_eval is not specified using default: 16384
Mixed Precision training with scaler: 1024 is enabled.
[10d05h56m42s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[10d05h56m42s][HUGECTR][INFO]: Algorithm search: ON
[10d05h56m43s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[10d05h56m43s][HUGECTR][INFO]: Initial seed is 2139207268
[10d05h56m43s][HUGECTR][INFO]: num_internal_buffers 1
[10d05h56m44s][HUGECTR][INFO]: num_internal_buffers 1
[10d05h56m44s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=165237
[10d05h57m03s][HUGECTR][INFO]: gpu0 start to init embedding of slot0 , slot_size=138493, key_offset=0, value_index_offset=0
[10d05h57m04s][HUGECTR][INFO]: gpu0 start to init embedding of slot1 , slot_size=26744, key_offset=138493, value_index_offset=138493
[10d05h57m04s][HUGECTR][INFO]: decay_start is not specified 